In [11]:
#uploading the data file
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
#importing inbuild libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import os
import gc
gc.enable()
from operator import itemgetter
from tqdm import tqdm
from scipy import optimize
from sklearn.metrics import mean_squared_error as MSE
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.gaussian_process import kernels
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")



In [13]:
#Taking log as a global variable
log = ""


In [14]:
#Establishing directoyires to read original as well as incomplete datasets
def read_subsets_and_original(BASE_PATH, ORIGINAL_BASE_PATH, dataset_name):
  sub_incomplete_dataset = os.listdir(BASE_PATH + dataset_name)
  print ("Found Total Subsets : ", len(sub_incomplete_dataset))
  original = pd.read_csv(ORIGINAL_BASE_PATH +'data12.csv', header=None)
  original = original.infer_objects()
  subsets = {}
  for each in tqdm(sub_incomplete_dataset, total=len(sub_incomplete_dataset)):
    subsets[each.split('.')[0]] = pd.read_csv(BASE_PATH + dataset_name + '/' + each, header=None)
  return subsets, original


In [15]:
#Calculate NRMS value (Formula according to the documentation)
def calculate_NRMS(y_true, y_pred):
  upper_values = y_pred - y_true
  #CHECK DOCUMENTATION ON https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html

  #ord = 'fro' means frobenius norm that is the euclidian distance
  upper_normed = np.linalg.norm(upper_values, ord='fro')
  lower_normed = np.linalg.norm(y_true, ord='fro')
  return upper_normed / lower_normed

In [16]:
from numpy.random.mtrand import shuffle
#Build GRNN Model
class GRNN(BaseEstimator, RegressorMixin):
    #Initializing all the elements
    def __init__(self, kernel='RBF', sigma=0.7, n_splits=5, calibration='warm_start', method='L-BFGS-B', bnds=(0, None), n_restarts_optimizer=0, seed = 42):
        self.kernel = kernel
        self.sigma = sigma
        self.n_splits = n_splits
        self.calibration = calibration
        self.method = method
        self.iterations = 0
        self.bnds = bnds
        self.n_restarts_optimizer = n_restarts_optimizer
        self.seed = seed
        
    def fit(self, X, y):

        # Check that X and y have correct shape
        # X, y = check_X_y(X, y)
        
        self.X_ = X
        self.y_ = y
        bounds = self.bnds
        
        np.seterr(divide='ignore', invalid='ignore')
         #Initializaing and establishing the cost function
        def cost(sigma_):
            kf = KFold(n_splits= self.n_splits)
            kf.get_n_splits(self.X_)
            cv_err = []
            for train_index, validate_index in kf.split(self.X_):
                X_tr, X_val = self.X_[train_index], self.X_[validate_index]
                y_tr, y_val = self.y_[train_index], self.y_[validate_index]
                Kernel_def_= getattr(kernels, self.kernel)(length_scale=sigma_)
                K_ = Kernel_def_(X_tr, X_val)
                # If the distances are very high/low, zero-densities must be prevented:
                K_ = np.nan_to_num(K_)
                psum_ = K_.sum(axis=0).T # Cumulate denominator of the Nadaraya-Watson estimator
                psum_ = np.nan_to_num(psum_)
                y_pred_ = (np.dot(y_tr.T, K_) / psum_)
                y_pred_ = np.nan_to_num(y_pred_)
                cv_err.append(MSE(y_val, y_pred_.T))
                break
            return cv_err[0] ## Mean error over the k splits                        
        
        #Establising the optimization function
        def optimization(x0_):
            rlog = ""
            if len(self.bnds) > 1:
              self.bnds = (self.bnds[0], )


            try:
              if len(x0_) > 1:
                x0_ = x0_[0]
            except:
              rlog = "x0_ is Good Enough"
               # print ("x0_", x0_)
            # print ("Bounds : ", self.bnds)
            opt = optimize.minimize(cost, x0_, method=self.method, bounds=self.bnds)
            if opt['success'] is True:
                opt_sigma = opt['x']
                opt_cv_error = opt['fun']
            else:
                opt_sigma = np.full(len(self.X_[0]), np.nan)
                opt_cv_error = np.inf
                pass
            return [opt_sigma, opt_cv_error]
        
        #Regulating and calibrating sigma
        def calibrate_sigma(self):
            x0 = np.asarray(self.sigma) # Starting guess (either user-defined or measured with warm start)
            if self.n_restarts_optimizer > 0:
                # print ("################################")    
                optima = [optimization(x0)]            
                #First optimize starting from theta specified in kernel
                optima = [optimization(x0)] 
                # # Additional runs are performed from log-uniform chosen initial bandwidths
                r_s = np.random.RandomState(self.seed)
                for iteration in range(self.n_restarts_optimizer): 
                    x0_iter = np.full(len(self.X_[0]), np.around(r_s.uniform(0,1), decimals=3))
                    optima.append(optimization(x0_iter))             
            elif self.n_restarts_optimizer == 0: 
                # print ("Running SAD ONE")    
                optima = [optimization(x0)]            
            else:
                raise ValueError('n_restarts_optimizer must be a positive int!')
            
            # Select sigma from the run minimizing cost
            cost_values = list(map(itemgetter(1), optima))
            self.sigma = optima[np.argmin(cost_values)][0]
            self.cv_error = np.min(cost_values) 
            return self
        global log
        if self.calibration is 'warm_start':
            log = log + 'Executing warm start...' + '/n'
            self.bnds = (bounds,)           
            x0 = np.asarray(self.sigma)
            optima = [optimization(x0)]            
            cost_values = list(map(itemgetter(1), optima))
            self.sigma = optima[np.argmin(cost_values)][0]
            log = log + 'Warm start concluded. The optimum isotropic sigma is ' + str(self.sigma) + '/n'
            self.sigma = np.full(len(self.X_[0]), np.around(self.sigma, decimals=3))
            self.bnds = (bounds,)*len(self.X_[0])
            # print ('Executing gradient search...')
            calibrate_sigma(self)
            log = log + 'Gradient search concluded. The optimum sigma is ' + str(self.sigma) + '/n'
        elif self.calibration is 'gradient_search':
            #print ('Executing gradient search...')
            self.sigma = np.full(len(self.X_[0]), self.sigma)
            self.bnds = (bounds,)*len(self.X_[0])
            calibrate_sigma(self)
            #print('Gradient search concluded. The optimum sigma is ' + str(self.sigma))
        else:
            pass
                   
        self.is_fitted_ = True
        # Return the regressor
        return self
   #Gathering all the above and predicting the values 
    def predict(self, X):
        
         # Check if fit had been called
        # check_is_fitted(self, ['X_', 'y_'])
        
        # Input validation
        X = check_array(X)
        
        Kernel_def= getattr(kernels, self.kernel)(length_scale=self.sigma)
        K = Kernel_def(self.X_, X)
        # If the distances are very high/low, zero-densities must be prevented:
        K = np.nan_to_num(K)
        psum = K.sum(axis=0).T # Cumulate denominator of the Nadaraya-Watson estimator
        psum = np.nan_to_num(psum)
        return np.nan_to_num((np.dot(self.y_.T, K) / psum))


In [17]:
#saga feature selection
def SAGA_FEATURE_SELECTION(X_train, y_train):
  model_logistic = Ridge(solver='saga')
  sel_model_logistic = SelectFromModel(estimator=model_logistic)
  X_train_sfm_l1 = sel_model_logistic.fit_transform(X_train.values, y_train.values)
  Indicator_columns = sel_model_logistic.get_support()
  return Indicator_columns #SAGA BASED FEATURE SELECTION

In [22]:
#Reading the data to run our model on
BASE = 'gdrive/My Drive/'
BASE_PATH = BASE + 'incomplete/'
ORIGINAL_BASE_PATH = BASE + 'complete_dataset/'
data = pd.read_excel(BASE + 'Table-NRMS.xlsx')
#data = pd.read_csv(BASE + 'List.csv')


In [23]:
NRMS_DICT = {}


In [ ]:
import time
begin_time = time.time()

for index, row in tqdm(data.iterrows(), total=data.shape[0]): #For Dataset
  subsets, original = read_subsets_and_original(BASE_PATH, ORIGINAL_BASE_PATH, row['Datasets']) # Get All Subsets and Original Dataset
  subset_names = list(subsets.keys())
  print(subset_names)
  #ITERATE OVER ALL SUBSETS OF A DATASET AND APPLY GRNN ON EACH ONE

  for each_subset_name in subset_names:

    #SELECTING A SUBSET
    selected_subset = subsets[each_subset_name]
    #print(selected_subset)


    new_prediction = np.zeros(shape=original.shape) #SAMPLE ARRAY TO SAVE PREDICTIONS
    #print(new_prediction.shape)
    #print(selected_subset.columns)
    new_prediction = pd.DataFrame(data = new_prediction, columns=selected_subset.columns)



    #COLUMNS ARRAY TO ITERATE
    all_cols = np.array(original.columns) 
    for each in tqdm(all_cols, total=len(all_cols)):



      #ONE COLUMN IN TEST AND OTHERS IN TRAINING
      train_cols = all_cols[all_cols != each] 
      test_col = each

      #CHECKING IF THERE ARE NULL VALUES IN OUR TEST COLUMNS
      nulls = selected_subset[each].isnull() 
     
      test_index = nulls[nulls == True].index
      train_index = nulls[nulls == False].index
    #  print(test_index.shape)
      #print(test_index.shape[0] / float(nulls.shape[0]))



      #IF THERE IS NO NULL VALUE THEN WE WONT APPLY GRNN
      if test_index.shape[0] == 0:
        new_prediction[each] = original[each].copy()
    

      else:

        #TRAIN GRNN ON INDEX WHERE THERE IS NO NULL AND PREDICT ON NULL VALUES
        custom_GRNN = GRNN()
        #print('in grnn')
        SAGA_BASED_FEATURES = SAGA_FEATURE_SELECTION(original[train_cols].loc[train_index], original[test_col].loc[train_index])
        #print(SAGA_BASED_FEATURES) #SAGA
    


        #Normalization
        normalizer = StandardScaler()

        train_X = original[train_cols[SAGA_BASED_FEATURES]].loc[train_index].values
        train_Y = original[test_col].loc[train_index].values
       # print(train_X)
        test_X = original[train_cols[SAGA_BASED_FEATURES]].loc[test_index].values
        #print(train_X, train_Y)

        normalizer.fit(train_X, train_Y)

        normalizer_train_X = normalizer.transform(train_X)
        normalizer_test_X = normalizer.transform(test_X)
        ############################################
        ############################################
        


        custom_GRNN.fit(normalizer_train_X, train_Y)
        #PREDICT
        prediction_smothened = custom_GRNN.predict(normalizer_test_X)
        #print(prediction_smothened)

        #FILL OUR SAVING ARRAY WITH PREDICTIONS
        new_prediction[each].loc[train_index] = selected_subset[each].loc[train_index]
        new_prediction[each].loc[test_index] = prediction_smothened
       # print(each_subset_name)
       # print(new_prediction)
        new_prediction.to_csv(BASE + "imputed_" + each_subset_name + ".csv", index=False)
        NRMSE = calculate_NRMS(original.values, new_prediction.values)
        print(NRMSE)
        log = log + "Done Smoothing of : " + each_subset_name + " with NRMS : " + str(NRMSE) + '/n/n/n'
        NRMS_DICT[each_subset_name] = NRMSE

end_time = time.time()
diff = end_time - begin_time
print(diff)
print(NRMS_DICT)
#print(log)
#df = pd.DataFrame(NRMS_DICT.values())
value = pd.DataFrame.from_dict(NRMS_DICT, orient='index')
#logs = log.split('/n')
#for each in logs:
#    x = each.split(' ')   
#    if len(x) > 0:
#        if x[0] == 'Done':
##            df['NRMS'] = x[8]
 #           df['Datasets'] = x[4]
 #          print(x[8], x[4])#

print(value)
value.to_csv('Table.csv')

  0%|          | 0/38 [00:00<?, ?it/s]

Found Total Subsets :  1



100%|██████████| 1/1 [00:00<00:00,  1.35it/s]


['Data_12_AG_10%']



  2%|▏         | 1/51 [01:19<1:06:28, 79.76s/it]

...


  3%|▎         | 1/38 [1:11:50<44:17:53, 4310.09s/it]

0.1939019999046496
Found Total Subsets :  1



100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


['Data_12_AG_20%']



  0%|          | 0/51 [00:00<?, ?it/s]